# Importing

In [2]:
#For data manipulation
import pandas as pd

#For web scraping
import bs4 as bs 
from bs4 import SoupStrainer

#Automating the scraping process
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#To use chrome as the web browser
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Fetching our dataset and preprocessing

In [21]:
df  = pd.read_csv('netflix_dataset.csv')
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


# Getting the urls

In [35]:
urlLst = []
myDict = {}
for i in range(len(df)):
    if df.loc[i, "type"] == 'Movie':
        url = 'https://www.rottentomatoes.com/m/'+df.loc[i, "title"].lower().replace(' ','_')+'/reviews?type=user'
    else:
        url = 'https://www.rottentomatoes.com/tv/'+df.loc[i, "title"].lower().replace(' ','_')+'/s01/reviews?type=user'
    #urlLst.append(url)
    myDict[url]={'Title': df.loc[i, "title"], 'Type' : df.loc[i,"type"]}

In [27]:
myDict

{'https://www.rottentomatoes.com/m/dick_johnson_is_dead/reviews?type=user': {'Title': 'Dick Johnson Is Dead',
  'Type': 'Movie'},
 'https://www.rottentomatoes.com/tv/blood_&_water/s01/reviews?type=user': {'Title': 'Blood & Water',
  'Type': 'TV Show'},
 'https://www.rottentomatoes.com/tv/ganglands/s01/reviews?type=user': {'Title': 'Ganglands',
  'Type': 'TV Show'},
 'https://www.rottentomatoes.com/tv/jailbirds_new_orleans/s01/reviews?type=user': {'Title': 'Jailbirds New Orleans',
  'Type': 'TV Show'},
 'https://www.rottentomatoes.com/tv/kota_factory/s01/reviews?type=user': {'Title': 'Kota Factory',
  'Type': 'TV Show'},
 'https://www.rottentomatoes.com/tv/midnight_mass/s01/reviews?type=user': {'Title': 'Midnight Mass',
  'Type': 'TV Show'},
 'https://www.rottentomatoes.com/m/my_little_pony:_a_new_generation/reviews?type=user': {'Title': 'My Little Pony: A New Generation',
  'Type': 'Movie'},
 'https://www.rottentomatoes.com/m/sankofa/reviews?type=user': {'Title': 'Sankofa',
  'Type': '

# Setting up the selenium driver for automation 

In [43]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options)

In [44]:
#This will hold each review's date
reviewTimeLst = []
#This will hold the star ratings for each review
starRatingLst = []
#This will hold each review content
reviewContentLst = []
#This list gives the movie/tv show title to which each review belongs
titleLst = []\
#The type of each title
typeLst = []

for url in myDict.keys():
    
    counter2 = 1
    try:
        name = myDict[url]['Title']
        titletype = myDict[url]['Type']
        driver.get(url)
    except:
        continue
    while True:
        if counter2 >40: break
        try:
            liLst = driver.find_elements(By.CLASS_NAME,"audience-reviews__item")
        except:
            break
        for item in liLst:
            try:
                date=item.find_element(By.CLASS_NAME,"audience-reviews__duration").text
                content=item.find_element(By.TAG_NAME,'p').text
            except:
                continue
            try:
                starRating = len(item.find_elements(By.CLASS_NAME,"star-display__filled"))+len(item.find_elements(By.CLASS_NAME,"star-display__half"))*0.5            
            except:
                starRating = 0
            reviewTimeLst.append(date)
            reviewContentLst.append(content)
            titleLst.append(name)
            starRatingLst.append(starRating)
            typeLst.append(titletype)
        try:    
            #clicking the next button to move to the next audience review page for scraping
            driver.find_element(By.CLASS_NAME,"prev-next-paging__wrapper").find_elements(By.TAG_NAME,"button")[-1].click()
        except:
            break
        counter2 += 1
    
    counter+=1
    

# Saving the data in a dataframe

In [45]:
titleDetails = {'Title': titleLst, 'Review': reviewContentLst, 'Date':reviewTimeLst, 'Rating':starRatingLst, 'Type':typeLst}  
       
reviewdf = pd.DataFrame(titleDetails) 
    

In [47]:
df2 = reviewdf[['Title', 'Review','Date','Rating']].drop_duplicates(keep='first')

In [49]:
#saving our output
df2.to_csv('RottenTomatoesReviews.csv') 